In [ ]:
import os
import cv2
import ast
import shutil
import random
import scipy.io
import numpy as np
import pandas as pd


In [ ]:

def calculate_optical_flow(img1_path, img2_path, b1, b2, output_file):

    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    
    roi1 = img1[y1:y1+h1, x1:x1+w1]
    
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    cropped_gray1 = gray1[y1:y1+h1, x1:x1+w1]
    cropped_gray2 = gray2[y2:y2+h2, x2:x2+w2]

    flow = cv2.calcOpticalFlowFarneback(cropped_gray1, cropped_gray2, None, 0.8, 3, 15, 3, 5, 1.2, 0)
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    angle = cv2.resize(angle, (cropped_gray1.shape[1], cropped_gray1.shape[0]), interpolation=cv2.INTER_LINEAR)

    hsv = np.zeros_like(cropped_gray1)
    hsv = np.stack((hsv, hsv, hsv), axis=-1)
    hsv[..., 1] = 255
    hsv[..., 0] = angle * 180 / np.pi / 2
    hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

    flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    flow_rgb = 0.033 * flow_rgb
    # print(flow_rgb.shape)
    flow_rgb = flow_rgb.astype(roi1.dtype)
    result = cv2.addWeighted(roi1, 1, flow_rgb, 2, 0)
    img1[y1:y1+h1, x1:x1+w1] = result
    cv2.imwrite(output_file+'roi.jpg',roi1)
    cv2.imwrite(output_file+'flow.jpg',flow_rgb)
    cv2.imwrite(output_file+'result.jpg', img1)

im1 = "D:/sampled_frames/clapping/S1_clapping_HD/segmented_images/S1_clapping_HD_32.jpg"
im2 = "D:/sampled_frames/clapping/S1_clapping_HD/segmented_images/S1_clapping_HD_53.jpg"
b1 = (917, 685, 128, 310)
b2 = (915, 685, 128, 310)
output_file = "D:/sampled_frames/clapping/S1_clapping_HD/wta_images/"

calculate_optical_flow(im1, im2, b1, b2, output_file)


In [ ]:
import pandas as pd
import ast
def read_annotations_csv(csv_file):
    df = pd.read_csv(csv_file)
    annotations = df['annotations']
    temp = []
    for i in range(len(annotations)):
        t1 = annotations[i]
        t2 = ast.literal_eval(t1)
        temp.append(t2)
        
    # Parse annotations and modify according to the specified criteria
    max_val_index3 = max([anno[2] for anno in temp])
    max_val_index4 = max([anno[3] for anno in temp])
    modified_annotations = [[anno[0], anno[1], max_val_index3, max_val_index4] for anno in temp]
    for i in range(len(modified_annotations)):
        df['annotations'][i] = modified_annotations[i]
    return df


def calculate_optical_flow(img1_path, img2_path, b1, b2):
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    
    roi1 = img1[y1:y1+h1, x1:x1+w1]
    
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    cropped_gray1 = gray1[y1:y1+h1, x1:x1+w1]
    cropped_gray2 = gray2[y2:y2+h2, x2:x2+w2]

    flow = cv2.calcOpticalFlowFarneback(cropped_gray1, cropped_gray2, None, 0.8, 3, 15, 3, 5, 1.2, 0)
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    angle = cv2.resize(angle, (cropped_gray1.shape[1], cropped_gray1.shape[0]), interpolation=cv2.INTER_LINEAR)

    hsv = np.zeros_like(cropped_gray1)
    hsv = np.stack((hsv, hsv, hsv), axis=-1)
    hsv[..., 1] = 255
    hsv[..., 0] = angle * 180 / np.pi / 2
    hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

    flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    flow_rgb = 0.033 * flow_rgb
    flow_rgb = flow_rgb.astype(roi1.dtype)
    return flow_rgb

def calculate_total_optical_flow(image_folder, annotations):
    total_flow_rgb = None
    for i in range(len(annotations) - 1):
        img1_path = os.path.join(image_folder, annotations['image_name'][i])
        img2_path = os.path.join(image_folder, annotations['image_name'][i + 1])
        flow_rgb = calculate_optical_flow(img1_path, img2_path, annotations['annotations'][i], annotations['annotations'][i+1])
        if total_flow_rgb is None:
            total_flow_rgb = flow_rgb
        else:
            total_flow_rgb += flow_rgb
    if total_flow_rgb is not None:
        # total_flow_rgb = cv2.multiply(total_flow_rgb, 0.033)  # Multiply by scalar using cv2.multiply
        total_flow_rgb = cv2.normalize(total_flow_rgb, None, 0, 255, cv2.NORM_MINMAX)  # Scale to 0-255 range
        total_flow_rgb = total_flow_rgb.astype(np.uint8)
    return total_flow_rgb


def read_annotations_csv2(csv_file):
    df = pd.read_csv(csv_file)
    annotations = df['annotations']
    temp = []
    for i in range(len(annotations)):
        t1 = annotations[i]
        t2 = ast.literal_eval(t1)
        temp.append(t2)
        
    # Parse annotations and modify according to the specified criteria
    # max_val_index3 = max([anno[2] for anno in temp])
    # max_val_index4 = max([anno[3] for anno in temp])
    # modified_annotations = [[anno[0], anno[1], max_val_index3, max_val_index4] for anno in temp]
    for i in range(len(temp)):
        df['annotations'][i] = temp[i]
    return df

def get_annotation_for_image(img_name, annotations):
    # Extract the index number from the image name
    index_number = int(img_name.split('_')[-1].split('.')[0])  # Extract the number after the last underscore
    
    # Check if the index exists in the annotations DataFrame
    if index_number in annotations['indices'].values:
        # Get the corresponding annotation
        annotation = annotations.loc[annotations['indices'] == index_number, 'annotations'].values[0]
        return annotation
    else:
        return None

def overlay_optical_flow(image_folder, annotations, result, output_folder):
    for img_name in os.listdir(image_folder):
        img_path = os.path.join(image_folder, img_name)
        img = cv2.imread(img_path)
        
        # Get the annotation for the current image
        annotation = get_annotation_for_image(img_name, annotations)
        
        if annotation is None:
            print(f"No annotation found for {img_name}")
            continue
        
        x, y, w, h = annotation
        roi = img[y:y+h, x:x+w]
        
        # Resize optical flow result to match the size of the ROI
        flow_overlay = cv2.resize(result, (w, h))
        
        # Blend optical flow visualization with the ROI
        blended_img = cv2.addWeighted(roi, 1.2, flow_overlay, 0.9, 0)
        
        # Replace the region in the original image with the blended result
        img[y:y+h, x:x+w] = blended_img
        
        # Write the image with overlay to the output folder
        output_path = os.path.join(output_folder, img_name)
        cv2.imwrite(output_path, img)

